In [1]:
import numpy as np
import os
import pandas as pd
import arcpy

from IPython.display import display, Markdown

# Import the required ArcGIS API for Python modules
import arcgis
from arcgis.gis import GIS
gis = arcgis.GIS()
from arcgis.geoanalytics import manage_data

import time

In [2]:
rtpSuffix       = 'RTP2023'

dirWork                    = os.getcwd()
dirInput                   = os.path.join(dirWork ,        r'input'                  )
dirATOdata                 = os.path.join(dirInput,        r'TDM_ATO_Output_RTP_v900')
dirIntermediate            = os.path.join(dirWork,         r'intermediate'           )
dirResults                 = os.path.join(dirWork,         r'results'                )
dirIntermediateTravelSheds = os.path.join(dirIntermediate, r'travelsheds'            )

print("Working Directory: "      + dirWork          )
print("Input Directory: "        + dirInput         )
print("Intermediate Directory: " + dirIntermediate  )
print("Results Directory: "      + dirResults       )

gdb_Process_name         = 'process_travelsheds.gdb'
gdb_Process              = os.path.join(dirIntermediate, 'process_travelsheds.gdb')
gdb_ATO_TravelSheds_name = 'TravelSheds_' + rtpSuffix + '.gdb'
gdb_ATO_TravelSheds      = os.path.join(dirResults, r'TravelSheds_' + rtpSuffix + '.gdb')

#name of TAZ Centroids
filenameTDMNodes             = os.path.join(dirATOdata, 'v9_SE50_Net50_Node.shp')

#name of TAZs
filenameTAZshp               = os.path.join(dirResults, 'TAZ_ATO_' + rtpSuffix + '.shp')

#utah shapefile
filenameUtahshp              = os.path.join(dirIntermediate, 'UtahSimple.shp')

#intermediate filenames
filenameTravelShedsForOneTAZData      = os.path.join(dirIntermediate, 'travelsheds\TAZShedsData.csv')
filenameTravelShedsForOneTAZJoin      = os.path.join(gdb_Process, 'TAZSheds_Join')
filenameTravelShedsForOneTAZDissolve  = os.path.join(gdb_Process, 'TAZSheds_Dissolve')
filenameTravelShedsForOneTAZPolylines = os.path.join(gdb_Process, 'TAZSheds_Polylines')
filenameTravelShedsForOneTAZMask      = os.path.join(gdb_Process, 'TAZSheds_Mask')

filenameTravelSheds                   = os.path.join(gdb_ATO_TravelSheds, rtpSuffix + '_TAZSheds')


def deleteIfExists(obj):
    if arcpy.Exists(obj): arcpy.Delete_management(obj)
              

#if results geodatabase doesn't exist, create it
print("Checking if " + gdb_Process_name + " exists...")
if not arcpy.Exists(os.path.join(dirIntermediate, gdb_Process_name)):
    print("Creating " + gdb_Process_name + "...")
    arcpy.management.CreateFileGDB(dirIntermediate, gdb_Process_name)
else:
    print(gdb_Process_name + " exists...")


listTDMs       = ['v9_SE23_Net23','v9_RTP_SE50_Net50']
listLayerNames = ['2023','2050']
listMode       = ['AUTO','TRAN']

Working Directory: e:\GitHub\ATO-Web-App\_dataprep
Input Directory: e:\GitHub\ATO-Web-App\_dataprep\input
Intermediate Directory: e:\GitHub\ATO-Web-App\_dataprep\intermediate
Results Directory: e:\GitHub\ATO-Web-App\_dataprep\results
Checking if process_travelsheds.gdb exists...
process_travelsheds.gdb exists...


In [3]:
# get list of TAZs in TAZ ATO, since some filtered out due to no hh job

sdfTAZATO = pd.DataFrame.spatial.from_featureclass(filenameTAZshp)
listValidTazs = sdfTAZATO['TAZID'].tolist()

In [4]:
skip = True

if not skip:

    df_TS_Summary = pd.DataFrame()

    i = -1

    #loop through tdm results from preprocessed data
    for tdm in listTDMs:
        
        i = i + 1 

        filenameTravelShed_Summary = os.path.join(dirIntermediateTravelSheds, tdm + '_TAZOpportunitySheds_Summary.csv')
        display(filenameTravelShed_Summary)

        df_TS_Summary_read = pd.read_csv(filenameTravelShed_Summary, index_col=False)
        
        df_TS_Summary_read['TDM'] = listLayerNames[i]
    
        df_TS_Summary = pd.concat([df_TS_Summary,df_TS_Summary_read])

    display(df_TS_Summary)
    df_TS_Summary.to_json(os.path.join(dirResults, 'TAZSheds_Summary_' + rtpSuffix + '.json'),orient='records',double_precision=2)


In [5]:
#if results geodatabase doesn't exist, create it
print("Checking if " + gdb_ATO_TravelSheds_name + " exists...")
if not arcpy.Exists(os.path.join(dirResults, gdb_ATO_TravelSheds_name)):
    print("Creating " + gdb_ATO_TravelSheds_name + "...")
    arcpy.management.CreateFileGDB(dirResults, gdb_ATO_TravelSheds_name)
else:
    print(gdb_ATO_TravelSheds_name + " exists...")
#print(AnalysisAreasInput)  

#debugging variables
debug = False
breakafter = 2

#variables
overwriteorappend = 'append'
#overwriteorappend = 'overwrite'
starttaz = 1
endtaz = 3546

#test data
#starttaz = 2080
#endtaz = 2100

try:
    lasttazfinished
except NameError:
    lasttazfinished = 0

#ONLY DEFINE lasttazfinished IF CLOSING AND REOPENING NOTEBOOK, OTHERWISE SHOULD BE PRESERVED IN NOTEBOOK
#lasttazfinished = 2794
    
display("Last TAZ finished: " + str(lasttazfinished))

#initialize variables... DON'T CHANGE THESE
i = -1
starttime = time.time()
endtime   = time.time()
breaknow = False


listTDMs       = ['v9_SE23_Net23','v9_RTP_SE50_Net50']
listLayerNames = ['2023','2050']
listMode       = ['AUTO','TRAN'] #



if (debug): display ("DEBUG ON -- BREAKING AFTER " + str(breakafter) + " ITERATIONS")

#loop through tdm results from preprocessed data
for tdm in listTDMs:
    
    if (breaknow==False):
        
        i = i + 1

        #filenameTravelShed_Summary = os.path.join(dirIntermediateTravelSheds, tdm + '_TAZOpportunitySheds_Summary.csv')
        #display(filenameTravelShed_Summary)
        
        #df_TS_Summary = pd.read_csv(filenameTravelShed_Summary, index_col=False)
        
        #df_TS_Summary.to_json(os.path.join(dirResults, "TAZSheds_Summary_" + listLayerNames[i] + ".json"),orient='records',double_precision=2)

        #loop through modes
        for mode in listMode:
            
            if (breaknow==False):

                filenameTravelShed_Mode = os.path.join(dirIntermediateTravelSheds, tdm + '_TAZOpportunitySheds_' + mode + '.csv')
                display(filenameTravelShed_Mode)

                df_TS = pd.read_csv(filenameTravelShed_Mode, index_col=False)

                df_TS_TAZs = df_TS.groupby(['I'], as_index=False).agg({"J":[np.size]})

                #initialize feature class
                filenameTravelSheds_TdmMode = filenameTravelSheds + '_' + listLayerNames[i] + '_' + mode

                if (overwriteorappend == 'overwrite'):
                    display("Deleting " + filenameTravelSheds_TdmMode + "...")
                    deleteIfExists(filenameTravelSheds_TdmMode)

                j = -1

                for index, row in df_TS_TAZs.iterrows():

                    tazid = row['I'][0]

                    if (tazid>=starttaz and tazid>lasttazfinished and tazid<=endtaz and breaknow==False and tazid in listValidTazs):

                        j = j + 1

                        if (debug): display("j: " + str(j))

                        if (debug): display("TAZ " + str(tazid))

                        starttime = time.time()

                        df_TS_OneTAZ = df_TS[df_TS['I']==tazid]
                        #display(df_TS_OneTAZ)

                        #create table with just sheds for one TAZ
                        if (debug): display("create csv...")
                        df_TS_OneTAZ.to_csv(filenameTravelShedsForOneTAZData,index=False)

                        #join to table created in last step
                        if (debug): display("join...")
                            
                        deleteIfExists(filenameTravelShedsForOneTAZJoin)
                        
                        joined_table =  arcpy.management.AddJoin(filenameTAZshp, 'TAZID', filenameTravelShedsForOneTAZData, 'J', 'KEEP_COMMON')
                        if (debug): display(joined_table)
                        arcpy.CopyFeatures_management(joined_table, filenameTravelShedsForOneTAZJoin) 

                        #dissolve on Shed value
                        if (debug): display("dissolve...")
                        deleteIfExists(filenameTravelShedsForOneTAZDissolve)
                        arcpy.management.Dissolve(filenameTravelShedsForOneTAZJoin, filenameTravelShedsForOneTAZDissolve, ["TAZShedsData_csv_I","TAZShedsData_csv_SHED"], "", "SINGLE_PART")

                        arcpy.AddField_management(filenameTravelShedsForOneTAZDissolve, "I"   , "SHORT")
                        arcpy.AddField_management(filenameTravelShedsForOneTAZDissolve, "SHED", "SHORT")

                        #for integer values, not sure how to calculate, so using string that gets changed by to integer due to field type
                        arcpy.CalculateField_management(filenameTravelShedsForOneTAZDissolve, "I",    '!TAZShedsData_csv_I!'     , "PYTHON3")
                        arcpy.CalculateField_management(filenameTravelShedsForOneTAZDissolve, "SHED", '!TAZShedsData_csv_SHED!'  , "PYTHON3")

                        arcpy.DeleteField_management(filenameTravelShedsForOneTAZDissolve, ["TAZShedsData_csv_I","TAZShedsData_csv_SHED"])

                        #create polylines (FUTURE USE?)
                        #deleteIfExists(filenameTravelShedsForOneTAZPolylines)
                        #arcpy.management.PolygonToLine(filenameTravelShedsForOneTAZDissolve, filenameTravelShedsForOneTAZPolylines, "IGNORE_NEIGHBORS")

                        deleteIfExists(filenameTravelShedsForOneTAZMask)

                        if (debug): display("create state mask...")
                        arcpy.analysis.SymDiff(filenameUtahshp, filenameTravelShedsForOneTAZDissolve, filenameTravelShedsForOneTAZMask)

                        arcpy.AddField_management(filenameTravelShedsForOneTAZMask, "I"   , "SHORT")
                        arcpy.AddField_management(filenameTravelShedsForOneTAZMask, "SHED", "SHORT")

                        arcpy.CalculateField_management(filenameTravelShedsForOneTAZMask, "I", '"' + str(tazid) + '"', "PYTHON3")

                        arcpy.DeleteField_management(filenameTravelShedsForOneTAZMask, ["FID_UtahSimple","OBJECTID","FID_TAZShedsDissolve","TAZShedsData_csv_I","TAZShedsData_csv_SHED"])

                        #if final layer does not exist copy, else append
                        if (debug): display("append sheds...")
                        if not arcpy.Exists(filenameTravelSheds_TdmMode):
                            arcpy.CopyFeatures_management(filenameTravelShedsForOneTAZDissolve, filenameTravelSheds_TdmMode)
                        else:
                            arcpy.management.Append(filenameTravelShedsForOneTAZDissolve, filenameTravelSheds_TdmMode)

                        # Create a field mappings object
                        field_mappings = arcpy.FieldMappings()

                        # Add field mappings for 'I' and 'SHED'
                        for field_name in ['I', 'SHED']:
                            # Create a field map for the current field
                            field_map = arcpy.FieldMap()
                            field_map.addInputField(filenameTravelSheds_TdmMode, field_name)

                            # Get the field properties from the field map
                            output_field = field_map.outputField
                            output_field.name = field_name
                            field_map.outputField = output_field

                            # Add the field map to the field mappings
                            field_mappings.addFieldMap(field_map)

                        if (debug): display("append mask to final layer...")
                        arcpy.management.Append(filenameTravelShedsForOneTAZMask, filenameTravelSheds_TdmMode, "NO_TEST", field_mappings)

                        endtime = time.time()

                        display("TAZ " + str(tazid) + " Elapsed Seconds: " + str((endtime - starttime)))

                        #keep track of tazid that has been completed, so the process can pick up after this one if broken
                        lasttazfinished = tazid
                        
                        #reset lasttazfinished after reaching the end taz
                        if (lasttazfinished == endtaz):
                            lasttazfinished = 0
                        
                        #conditions to break if debug mode is on
                        if (j>=breakafter and debug==True):
                            breaknow = True;
                            display("BREAKING!")

                #reset lasttazfinished
                lasttazfinished = 0

Checking if TravelSheds_RTP2023.gdb exists...
TravelSheds_RTP2023.gdb exists...


'Last TAZ finished: 2794'

'e:\\GitHub\\ATO-Web-App\\_dataprep\\intermediate\\travelsheds\\v9_SE23_Net23_TAZOpportunitySheds_TRAN.csv'

'TAZ 2795 Elapsed Seconds: 9.351197957992554'

'TAZ 2796 Elapsed Seconds: 6.685186386108398'

...

'TAZ 3461 Elapsed Seconds: 14.922548532485962'

'TAZ 3462 Elapsed Seconds: 14.998746871948242'

'TAZ 3463 Elapsed Seconds: 15.037339925765991'

'TAZ 3502 Elapsed Seconds: 15.116442203521729'

'TAZ 3503 Elapsed Seconds: 15.09951114654541'

In [6]:
lasttazfinished

0